In [1]:
#Import modules and dependecies
import pandas as pd
import numpy as np
import plotly.express as px

from dash import html, Dash, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from dash_bootstrap_templates import load_figure_template

In [6]:
#Load dataframes
df1 = pd.read_csv('datasets/final_df1.csv')
df1_multi = pd.read_csv('datasets/final_df1.csv', index_col=[0,1])
df2 = pd.read_csv('datasets/final_df2.csv', index_col=0)

In [8]:
#Create the map